In [4]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Dropout, GlobalMaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model

vgg = VGG16(input_shape=(720,1280,3), weights='imagenet', include_top=False)
for layer in vgg.layers:
    layer.trainable = False
x = Flatten()(vgg.output)

prediction = Dense(5, activation='softmax')(x)

modelvgg = Model(inputs=vgg.input, outputs=prediction)
modelvgg.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 720, 1280, 3)]    0         
                                                                 
 block1_conv1 (Conv2D)       (None, 720, 1280, 64)     1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 720, 1280, 64)     36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 360, 640, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 360, 640, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 360, 640, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 180, 320, 128)     0   

In [2]:
save_path='vgg.h5'
modelvgg.save(save_path)

In [3]:
from tensorflow.keras.models import load_model
model = load_model("./vgg.h5")
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [13]:
import numpy as np
array = np.random.rand(1,224,224,3)
model(array)

<tf.Tensor: shape=(1, 5), dtype=float32, numpy=
array([[0.25954542, 0.3610852 , 0.2257354 , 0.0245275 , 0.1291065 ]],
      dtype=float32)>

In [16]:
def build_model(): 
    input_layer = Input(shape=(224,224,3))
    
    vgg = VGG16(include_top=False)(input_layer)

    # Classification Model  
    f1 = GlobalMaxPooling2D()(vgg)
    class1 = Dense(2048, activation='relu')(f1)
    class2 = Dense(5, activation='softmax')(class1)
    
    # Bounding box model
    f2 = GlobalMaxPooling2D()(vgg)
    regress1 = Dense(2048, activation='relu')(f2)
    regress2 = Dense(4, activation='softmax')(regress1)
    
    model = Model(inputs=input_layer, outputs=[class2, regress2])
    return model

model = build_model()
model.save("vgg16_od.h5")

/home/uraninjo/.local/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [20]:
od_model = load_model("./vgg16_od.h5")
rslt = model.predict()
rslt

1/1 [==============================] - 0s 74ms/step


[array([[0.11646918, 0.23042744, 0.14835194, 0.3552706 , 0.1494808 ]],
       dtype=float32),
 array([[0.43403637, 0.22089133, 0.16560455, 0.17946774]], dtype=float32)]

In [28]:
result_dict = {"classifications" : rslt[0].tolist(),
                "bbox_informations" : rslt[1].tolist()}
result_dict 



{'classifications': [[0.11646918207406998,
   0.23042744398117065,
   0.14835193753242493,
   0.3552705943584442,
   0.14948080480098724]],
 'bbox_informations': [[0.43403637409210205,
   0.2208913266658783,
   0.16560454666614532,
   0.17946773767471313]]}

In [5]:
import cv2
model = load_model("./vgg.h5")
data = cv2.imread("deneme.png")
data = tf.expand_dims(data, axis = 0)
results = model.predict(data).tolist()

1/1 [==============================] - 2s 2s/step


In [6]:
results

[[0.9999991655349731,
  1.6644538902710337e-07,
  6.650739692304342e-07,
  4.627938149078057e-15,
  1.2420334982721215e-08]]

In [29]:
import json
json.dumps(result_dict)

'{"classifications": [[0.11646918207406998, 0.23042744398117065, 0.14835193753242493, 0.3552705943584442, 0.14948080480098724]], "bbox_informations": [[0.43403637409210205, 0.2208913266658783, 0.16560454666614532, 0.17946773767471313]]}'

In [17]:
import torch
from torchvision.models import resnet50, ResNet50_Weights
from torch import nn
import numpy as np
model = resnet50()
model.fc = nn.Linear(model.fc.in_features, 4, bias=True)

torch.save(model, "torch_resnet50.h5")
t_model = torch.load("./torch_resnet50.h5")
t_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [18]:
t_model = torch.load("./torch_resnet50.h5")
array = np.random.rand(3, 224, 224)
tensor = torch.tensor(array).unsqueeze(0).float()
tensor.shape

torch.Size([1, 3, 224, 224])

In [9]:
DEVICE = "cuda" 

In [12]:
tensor = tensor.to(DEVICE)
t_model = t_model.to(DEVICE)
results = t_model(tensor)
results

tensor([[-0.7336,  0.0754, -0.6357,  0.4909]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [14]:
torch.Tensor.tolist(results)

[[-0.7336052656173706,
  0.07539453357458115,
  -0.6356832981109619,
  0.4908992052078247]]

In [21]:
model = torch.load("./torch_resnet50.h5").to(DEVICE)
data = cv2.imread("deneme.png")
data = np.transpose(data, (2, 0, 1))
data = torch.tensor(data).unsqueeze(0).float().to(DEVICE)
print(data.shape)
results = model(data)
results = torch.Tensor.tolist(results)
results

torch.Size([1, 3, 720, 1280])


[[0.33910462260246277,
  0.03539729118347168,
  0.0671568363904953,
  0.542465329170227]]